In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image

def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1

def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 40, 0, img)
    height, _ = img.shape

    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]

    return img_copy[row_ind][:, col_ind]


train_csv_file = '/Volumes/图图/CSAW-M/labels/CSAW-M_train.csv'
test_csv_file = '/Volumes/图图/CSAW-M/labels/CSAW-M_test.csv'
image_dir = '/Volumes/图图/CSAW-M/images/preprocessed'
output_dir = 'Benchmark/CSAW-M' 


train_df = pd.read_csv(train_csv_file, delimiter=';')
test_df = pd.read_csv(test_csv_file, delimiter=';')
combined_df = pd.concat([train_df, test_df], ignore_index=True)

def process_and_save(df):
    for index, row in df.iterrows():
        filename = row['Filename']


        image_path = os.path.join(image_dir, filename.split('_')[0], filename)
        

        try:
            image = Image.open(image_path)
            image_array = np.array(image)
            processed_image = ExtractBreast(image_array)
            
            case_output_dir = os.path.join(output_dir, filename.split('.')[0])
            os.makedirs(case_output_dir, exist_ok=True)
            output_image_path = os.path.join(case_output_dir, 'img.jpg')
            Image.fromarray(processed_image).save(output_image_path)
            
            print(f'Processed and saved: {output_image_path}')
        except Exception as e:
            print(f'Error processing {filename}: {str(e)}')

# 处理并保存所有数据
process_and_save(combined_df)
print("All data processing completed!")